###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 1: Pose classification with naive Bayes


**Student ID(s):**     1039169, 1044793


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
# Load library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Create URL
test_csv = "~/pose-classification-with-naive-bayes/COMP30027_2021_assignment1_data/test.csv"
train_csv = "~/pose-classification-with-naive-bayes/COMP30027_2021_assignment1_data/train.csv"
# Load Dataset 
test_df = pd.read_csv(test_csv, header = None)
train_df = pd.read_csv(train_csv, header = None)

# Duplicate Dataset
new_test_df = test_df.copy()
new_train_df = train_df.copy()

In [2]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(df):
    for column in df.columns[1:]:
        median = df[column].median()
        # Replace missing value (9999) with median for each column
        df[column] = np.where(df[column] == 9999, median, df[column])
    return df

In [3]:
# Preprocess testing data
new_test_df = preprocess(new_test_df)
new_test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,bridge,126.8358,99.92750,-2.0408,-0.69595,47.5551,-9.78480,7.377900,-65.10040,-62.3788,...,-14.5613,67.9703,76.2820,-30.33920,-41.2163,23.31460,57.56250,-24.6930,61.5094,-34.0565
1,bridge,4.5905,1.32125,-2.0408,-0.69595,46.6380,63.16715,38.160800,-43.07530,-47.0571,...,62.0490,67.9703,76.2820,63.26315,57.8452,-1.14960,31.49560,-11.7053,-4.8259,-13.8147
2,bridge,135.8192,99.76150,-2.0408,-0.69595,48.2256,-14.18380,16.306900,-131.97110,-153.9583,...,-5.0926,67.9703,76.2820,-38.72970,-48.6293,50.00130,86.12220,-56.0617,-25.4718,-59.5740
3,bridge,-95.5423,-42.97660,-31.4373,-50.12830,46.6380,63.16715,49.972800,-12.65285,-9.1416,...,7.6738,-42.7047,-87.5917,63.26315,57.8452,44.55740,-10.81735,-55.4993,41.6346,-59.5740
4,bridge,4.5905,-3.63780,-2.0408,-0.69595,16.4145,-12.77660,0.945965,-12.65285,-9.1416,...,71.5469,67.9703,76.2820,2.39560,-73.9424,23.39835,-10.81735,-55.4993,-25.4718,-59.5740


In [4]:
# Preprocess training data 
new_train_df = preprocess(new_train_df)
new_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,bridge,54.8598,33.01660,-19.5182,-0.14849,8.9676,-33.5934,-63.25060,-13.6323,-6.6108,...,14.5333,65.242100,73.0468,-30.5872,-57.6515,41.6003,-41.1442,-93.2785,-35.9018,-93.2520
1,bridge,2.7613,0.72352,-19.5182,-0.14849,46.9342,61.5274,0.47038,-13.6323,-6.6108,...,60.6513,65.242100,73.0468,64.8649,70.8083,17.8000,-41.1442,-93.2785,-35.9018,-93.2520
2,bridge,-87.4883,-38.94670,32.8427,93.59230,46.9342,61.5274,0.47038,-13.6323,-6.6108,...,-5.4110,0.091912,-3.8473,64.8649,70.8083,17.8000,-41.1442,-93.2785,-35.9018,-93.2520
3,bridge,-140.2714,-107.90450,-19.5182,-0.14849,46.9342,61.5274,-23.90420,65.4627,76.3410,...,-25.8142,65.242100,73.0468,64.8649,70.8083,28.4441,49.1976,-44.1205,38.9266,-37.7665
4,bridge,171.6807,72.98430,-19.5182,-0.14849,-56.6202,-167.9182,-20.12660,-13.6323,-6.6108,...,2.1167,65.242100,73.0468,-38.3855,-28.2247,38.5907,-41.1442,-93.2785,-35.9018,-93.2520


In [5]:
# Seperate class by feature_train, feature_test, feature_train, feature_test
feature_train = new_train_df.iloc[:,1:]
y_train = new_train_df[0]

feature_test = new_test_df.iloc[:,1:]
y_test = new_test_df[0]

In [6]:
# Find out the prior probability of each pose
target_count = 0
target_dict = dict()
for target in np.unique(y_train):
    target_count = sum(y_train == target)
    target_dict[target] = target_count / len(y_train)

In [39]:
"""x = list(feature_train.min())[0] + 1
# Equal Width Discretisation
width = (list(feature_train.max())[0] - list(feature_train.min())[0]) / 5

ordinal_feat = feature_train.copy()
print(ordinal_feat)
col = len(feature_train.columns)
row = len(feature_train)
i = 1
while i <= col:
    for value in feature_train[1]:
        
        # Put each data points into 5 buckets
        j = i - 1
        if x >= list(feature_train.min())[j] and x <= (list(feature_train.min())[j] + width):
            x = 1
        elif (x > list(feature_train.min())[j] + width) and x <= (list(feature_train.min())[j] + width*2):
            x = 2
        elif (x > list(feature_train.min())[j] + width*2) and x <= (list(feature_train.min())[j] + width*3):
            x = 3
        elif (x > list(feature_train.min())[j] + width*3) and x <= (list(feature_train.min())[j] + width*4):
            x = 4
        else:
            x = 5
    i += 1"""


'x = list(feature_train.min())[0] + 1\n# Equal Width Discretisation\nwidth = (list(feature_train.max())[0] - list(feature_train.min())[0]) / 5\n\nordinal_feat = feature_train.copy()\nprint(ordinal_feat)\ncol = len(feature_train.columns)\nrow = len(feature_train)\ni = 1\nwhile i <= col:\n    for value in feature_train[1]:\n        \n        # Put each data points into 5 buckets\n        j = i - 1\n        if x >= list(feature_train.min())[j] and x <= (list(feature_train.min())[j] + width):\n            x = 1\n        elif (x > list(feature_train.min())[j] + width) and x <= (list(feature_train.min())[j] + width*2):\n            x = 2\n        elif (x > list(feature_train.min())[j] + width*2) and x <= (list(feature_train.min())[j] + width*3):\n            x = 3\n        elif (x > list(feature_train.min())[j] + width*3) and x <= (list(feature_train.min())[j] + width*4):\n            x = 4\n        else:\n            x = 5\n    i += 1'

In [33]:
# Put each data points into 5 buckets
"""if x >= list(feature_train.min())[0] and x <= (list(feature_train.min())[0] + width):
    x = 1
elif (x > list(feature_train.min())[0] + width) and x <= (list(feature_train.min())[0] + width*2):
    x = 2
elif (x > list(feature_train.min())[0] + width*2) and x <= (list(feature_train.min())[0] + width*3):
    x = 3
elif (x > list(feature_train.min())[0] + width*3) and x <= (list(feature_train.min())[0] + width*4):
    x = 4
else:
    x = 5
print(feature_train.min())"""

1     -565.1104
2     -267.8630
3     -343.5755
4     -582.6049
5     -191.4188
6     -266.6518
7    -1443.1126
8    -1356.7781
9    -1001.1598
10    -343.3609
11    -336.1995
12    -259.8218
13    -194.9991
14    -153.1038
15    -272.0539
16    -187.2272
17    -230.4518
18    -114.5587
19    -151.7837
20    -345.3880
21    -201.1122
22   -3483.9990
dtype: float64


In [9]:
# This function should calculate prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train():
    return

In [10]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict():
    return

In [11]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate():
    return

## Questions 


If you are in a group of 1, you will respond to **two** questions of your choosing.

If you are in a group of 2, you will respond to **four** questions of your choosing.

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer should be submitted separately as a PDF.

### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

### Q2
The Gaussian naıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in this dataset? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the classifier’s predictions.

### Q3
Implement a kernel density estimate (KDE) naive Bayes classifier and compare its performance to the Gaussian naive Bayes classifier. Recall that KDE has kernel bandwidth as a free parameter -- you can choose an arbitrary value for this, but a value in the range 5-25 is recommended. Discuss any differences you observe between the Gaussian and KDE naive Bayes classifiers. (As with the Gaussian naive Bayes, this KDE naive Bayes implementation should be your own and should not just call a pre-existing function.)

### Q4
Instead of using an arbitrary kernel bandwidth for the KDE naive Bayes classifier, use random hold-out or cross-validation to choose the kernel bandwidth. Discuss how this changes the model performance compared to using an arbitrary kernel bandwidth.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

### Q6
Engineer your own pose features from the provided keypoints. Instead of using the (x,y) positions of keypoints, you might consider the angles of the limbs or body, or the distances between pairs of keypoints. How does a naive Bayes classifier based on your engineered features compare to the classifier using (x,y) values? Please note that we are interested in explainable features for pose recognition, so simply putting the (x,y) values in a neural network or similar to get an arbitrary embedding will not receive full credit for this question. You should be able to explain the rationale behind your proposed features. Also, don't forget the conditional independence assumption of naive Bayes when proposing new features -- a large set of highly-correlated features may not work well.